In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.3 MB/s eta 0:00:00


In [4]:
pip install tensorflow datasets matplotlib

In [5]:
import tensorflow as tf
import numpy as np
from datasets import load_dataset

**Load smart contract dataset from HuggingFace**

In [6]:
big_multilabel_dataset = load_dataset(
    path="mwritescode/slither-audited-smart-contracts",
    name="big-multilabel",
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.12k [00:00<?, ?B/s]

slither-audited-smart-contracts.py:   0%|          | 0.00/8.00k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/19.9k [00:00<?, ?B/s]

contracts0.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

...

data%2Fbig-splits.csv:   0%|          | 0.00/4.97M [00:00<?, ?B/s]

data%2Fsmall-splits.csv:   0%|          | 0.00/659k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/79641 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10861 [00:00<?, ? examples/s]

Clean Data

In [7]:
def clean_data(dataset):
  cleaned_data = []
  for data in dataset:
    # clean the bytecode and the 4 output that represents if the contract is safe
    if (len(data['bytecode']) > 4):
      new_slither_output = []
      for output in data['slither']:
         if (output > 4):
           new_slither_output.append(output - 1)
         elif (output < 4):
           new_slither_output.append(output)
      data['slither']=new_slither_output
      cleaned_data.append(data)
  return cleaned_data

cleaned_training_data = clean_data(big_multilabel_dataset["train"])
cleaned_validation_data = clean_data(big_multilabel_dataset["validation"])
cleaned_test_data = clean_data(big_multilabel_dataset["test"])

len(cleaned_training_data), len(cleaned_validation_data), len(cleaned_test_data)

(79414, 10831, 15921)

In [8]:
def split_text_into_chars(text, length):
  return " ".join([text[i:i+length] for i in range(0, len(text), length)])

train_bytecode = [split_text_into_chars(data['bytecode'],2) for data in cleaned_training_data]
test_bytecode = [split_text_into_chars(data['bytecode'],2) for data in cleaned_test_data]
val_bytecode = [split_text_into_chars(data['bytecode'],2) for data in  cleaned_validation_data]

In [9]:
# Convert labels to binary vectors
import numpy as np

def labels_to_binary(y, num_labels):
  """
  Converts the labels into binary format depending on the total number of labels,
  for example: y = [1,4], num_labels = 5, y_binary = [0,1,0,0,1,0]
  """
  y_binary = np.zeros((len(y), num_labels), dtype=float)
  for i, label_indices in enumerate(y):
      y_binary[i, label_indices] = 1
  return y_binary

# Extract 'slither' values from cleaned datasets
training_slither = [data['slither'] for data in cleaned_training_data] # Define training_slither by extracting 'slither' from cleaned_training_data
validation_slither = [data['slither'] for data in cleaned_validation_data] # Define validation_slither by extracting 'slither' from cleaned_validation_data
test_slither = [data['slither'] for data in cleaned_test_data] # Define test_slither by extracting 'slither' from cleaned_test_data


num_classes = len(np.unique(np.concatenate(training_slither)))

train_labels_binary = labels_to_binary(training_slither, num_classes)
valid_labels_binary = labels_to_binary(validation_slither, num_classes)
test_labels_binary = labels_to_binary(test_slither, num_classes)

def transform_labels_to_dict(labels_binary):
  labels_dict = {}
  for index in range(num_classes):
    labels_dict[f'{index}'] = []

  for labels in labels_binary:
    for index, label in enumerate(labels):
      labels_dict[f'{index}'].append(label)
  return labels_dict


validation_dict = transform_labels_to_dict(valid_labels_binary)
train_dict = transform_labels_to_dict(train_labels_binary)
test_dict = transform_labels_to_dict(test_labels_binary)

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_bytecode, train_dict)).batch(32).prefetch(tf.data.AUTOTUNE)
validation_dataset = tf.data.Dataset.from_tensor_slices((val_bytecode, validation_dict)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_bytecode, test_dict)).batch(32).prefetch(tf.data.AUTOTUNE)

**Create the TextVectorizer layer**

In [11]:
max_tokens = 10000 # Define the maximum number of tokens
output_seq_len = 250  # Define the output sequence length

text_vectorizer = tf.keras.layers.TextVectorization(
    split="whitespace",
    max_tokens=max_tokens, # Use the defined max_tokens
    output_sequence_length=output_seq_len # Use the defined output_seq_len
)

text_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(train_bytecode).batch(32).prefetch(tf.data.AUTOTUNE))

bytecode_vocab = text_vectorizer.get_vocabulary()
print(f"Number of different characters in vocab: {len(bytecode_vocab)}")
print(f"5 most common characters: {bytecode_vocab[:5]}")
print(f"5 least common characters: {bytecode_vocab[-5:]}")

Number of different characters in vocab: 259
5 most common characters: ['', '[UNK]', 'ff', '60', '00']
5 least common characters: ['be', 'bf', 'f6', 'fb', '0x']


**Create the Embedding layer**

In [12]:
embedding_layer = tf.keras.layers.Embedding(
    input_dim=len(bytecode_vocab),
    input_length=output_seq_len,
    output_dim=128,
    mask_zero=True, # Conv layers do not support masking but RNNs do
    name="embedding_layer"
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


**Create the model**

In [13]:
# Create input layer
from tensorflow.keras import layers # Import layers from tensorflow.keras
inputs = layers.Input(shape=(1,), dtype=tf.string)

# Create vectorization layer
x = text_vectorizer(inputs)

# Create embedding layer
x = embedding_layer(x)

# Create the LSTM layer
x = layers.GRU(units = 64, activation='tanh', return_sequences=True)(x)
x = layers.GRU(units = 32, activation='tanh')(x)
x = layers.Dropout(rate=0.2)(x)
x = layers.Dense(32, activation='relu')(x)

# Create the output layer
outputs = []
for index in range(num_classes):
  output = layers.Dense(1, activation="sigmoid", name=f'{index}')(x)
  outputs.append(output)

model_2 = tf.keras.Model(inputs = inputs, outputs = outputs, name="model_2")

**Compile the model**

In [14]:
losses={}
metrics={}
for index in range(num_classes):
  losses[f'{index}'] = "binary_crossentropy"
  metrics[f'{index}'] = ['accuracy']

model_2.compile(loss=losses, optimizer=tf.keras.optimizers.Adam(learning_rate=1e-03), metrics=metrics)

**Fit the model**

In [15]:
history_1 = model_2.fit(train_dataset,
                        epochs=20,
                        validation_data=validation_dataset,
                        callbacks=[
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                                        patience=5),
                                   tf.keras.callbacks.ModelCheckpoint(filepath=f"model_experiments/model_2.keras",
                                                                      monitor='val_loss',
                                                                      verbose=0,
                                                                      save_best_only=True)
                                  ])

Epoch 1/20
2482/2482 ━━━━━━━━━━━━━━━━━━━━ 131s 50ms/step - 0_accuracy: 0.8544 - 0_loss: 0.3797 - 1_accuracy: 0.8353 - 1_loss: 0.4072 - 2_accuracy: 0.7486 - 2_loss: 0.5107 - 3_accuracy: 0.7916 - 3_loss: 0.4427 - 4_accuracy: 0.7408 - 4_loss: 0.5186 - loss: 2.2589 - val_0_accuracy: 0.8879 - val_0_loss: 0.2917 - val_1_accuracy: 0.8622 - val_1_loss: 0.3414 - val_2_accuracy: 0.7628 - val_2_loss: 0.4736 - val_3_accuracy: 0.8489 - val_3_loss: 0.3480 - val_4_accuracy: 0.8178 - val_4_loss: 0.4144 - val_loss: 1.8700 - learning_rate: 0.0010
Epoch 2/20
2482/2482 ━━━━━━━━━━━━━━━━━━━━ 126s 51ms/step - 0_accuracy: 0.8907 - 0_loss: 0.2889 - 1_accuracy: 0.8672 - 1_loss: 0.3313 - 2_accuracy: 0.7936 - 2_loss: 0.4246 - 3_accuracy: 0.8694 - 3_loss: 0.3044 - 4_accuracy: 0.8297 - 4_loss: 0.3556 - loss: 1.7048 - val_0_accuracy: 0.9050 - val_0_loss: 0.2582 - val_1_accuracy: 0.8779 - val_1_loss: 0.3155 - val_2_accuracy: 0.7967 - val_2_loss: 0.4134 - val_3_accuracy: 0.8599 - val_3_loss: 0.3155 - val_4_accuracy: 0

**Predictions**

In [16]:
model_2 = tf.keras.models.load_model(filepath="model_experiments/model_2.keras")
model_2_preds_probs = model_2.predict(test_dataset)

498/498 ━━━━━━━━━━━━━━━━━━━━ 17s 34ms/step


In [21]:
def convert_preds_probs_to_preds(preds_probs):
    preds = []
    for pred_prob in preds_probs:
        converted_pred_prob = [1 if value[0] >= 0.5 else 0 for value in pred_prob]
        preds.append(converted_pred_prob)
    preds_dict = {}
    for index in range(len(preds)):
        preds_dict[f'{index}'] = preds[index]
    return preds_dict

model_2_preds = convert_preds_probs_to_preds(model_2_preds_probs)

In [22]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.

    Args:
    y_true: true labels in the form of a 1D array
    y_pred: predicted labels in the form of a 1D array

    Returns:
    A dictionary of accuracy, precision, recall, f1-score.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and f1 score using "weighted average"
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="weighted"
    )
    model_results = {
        "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1,
    }
    return model_results

def combine_results(y_true, y_pred, test_dict, model_2_preds, num_classes):
    results = {}
    for index in range(num_classes):
        results[f'{index}'] = calculate_results(
            y_true=test_dict[f'{index}'], y_pred=model_2_preds[f'{index}']
        )
    return results

In [24]:
import pandas as pd

results = combine_results(y_true=test_dict, y_pred=model_2_preds,
                          test_dict=test_dict, model_2_preds=model_2_preds,
                          num_classes=len(test_dict))

# Convert results dictionary to a DataFrame
results_df = pd.DataFrame(results)
print(results_df)

                   0          1          2          3          4
accuracy   93.141134  90.660134  87.010866  89.202940  88.254507
precision   0.928368   0.904168   0.867099   0.892336   0.882846
recall      0.931411   0.906601   0.870109   0.892029   0.882545
f1          0.926950   0.896839   0.864075   0.888595   0.882263


In [25]:
import pickle

# Save the trained model
model_2.save("vulnerability_detection_model.h5")

# Save the TextVectorizer for preprocessing
with open("text_vectorizer.pkl", "wb") as f:
    pickle.dump(text_vectorizer, f)
